In [ ]:
###############################################################
# @title : ESG 데이터 수집 최종 코드
# @date : 2023-10-10
#       : 2023-10-15
# @version : 1.4
###############################################################
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome() #또는 driver = webdriver.Chrome('chromedriver.exe')
# driver.implicitly_wait(5) # 페이지 활성화를 확인을 위해 최대 5초 wait

# 페이지 가져오기(이동)
driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
time.sleep(10)

# 연도
year_list = ['2020','2021','2022','2023']
# 연도별 기업수
ent_list = [758, 778, 791, 791]
# 결과 저장 리스트
tables = []

# 연도 element 선택 
year_row = driver.find_element(By.TAG_NAME,'select')

# 연도 리스트 반복
for cnt, year in enumerate(year_list):
    print(str(ent_list[cnt]) + '-' + year)
    
    year_row.send_keys(year) # 연도 값 지정

    search_btn = driver.find_element(By.CLASS_NAME,'sch_btn') # 조회 버튼
    search_btn.click() # 조회 버튼 클릭
    time.sleep(3)
    
    # 연도별 첫페이지 데이터 수집
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    div=soup.select_one("div#grade_table")
    table=pd.read_html(str(div))[0]
    
    # 리스트에 결과 추가 
    tables.append(table)
    
    # 페이지 수만큼 반복
    for i in range(int(ent_list[cnt]/10)):
        next_btn = driver.find_element(By.CSS_SELECTOR,'ul > li.next > a') # 다음 버튼
        
        wait = WebDriverWait(driver, timeout=300)
        wait.until(lambda d : search_btn.is_enabled())
        
        next_btn.click() # 다음 버튼 클릭
        
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        div=soup.select_one("div#grade_table")
        table=pd.read_html(str(div))[0]
        tables.append(table)
        # print(table)
        # print('-' + str(i+1), end='/')
        time.sleep(2)

    print(len(tables))
driver.close()

print(len(tables))

In [ ]:
# 원본 데이터 저장
import pickle

with open('esg_raw_data.pickle','wb') as fw:
    pickle.dump(tables,fw)

In [ ]:
##################################################################################################################################
# 데이터 전처리
##################################################################################################################################

In [5]:
# 원본데이터 읽기
import pickle
with open('esg_raw_data.pickle','rb') as fr:
    esg_raw_data_list = pickle.load(fr)

In [6]:
esg_raw_data_list[313]

회사명     KCGS                            한국ESG 연구소  \
                  회사명     종합등급     환경     사회     지배구조     한국ESG 연구소   
0      효성중공업 2023 N N   KCGS A   환경 A  사회 A+  지배구조 B+   한국ESG 연구소 -   
1     효성첨단소재 2023 N N   KCGS A  환경 B+  사회 A+   지배구조 A   한국ESG 연구소 A   
2      효성티앤씨 2023 N N   KCGS A   환경 A  사회 A+  지배구조 B+   한국ESG 연구소 A   
3       효성화학 2023 N N   KCGS A   환경 A  사회 A+  지배구조 B+   한국ESG 연구소 -   
4         후성 2023 N N   KCGS B   환경 B  사회 B+   지배구조 B  한국ESG 연구소 B+   
5      휠라홀딩스 2023 N N  KCGS B+   환경 B   사회 A  지배구조 B+   한국ESG 연구소 A   
6  휴니드테크놀러지스 2023 N N   KCGS D   환경 D   사회 C   지배구조 C   한국ESG 연구소 -   
7        휴비스 2023 N N  KCGS B+  환경 B+   사회 A  지배구조 B+   한국ESG 연구소 -   
8        휴스틸 2023 N N   KCGS D   환경 D   사회 D   지배구조 C   한국ESG 연구소 -   
9       흥국화재 2023 N N  KCGS B+   환경 C  사회 B+  지배구조 B+   한국ESG 연구소 -   

     서스틴 베스트     Moody's    MSCI     S&P 지속가능경영 보고서공시여부 Unnamed: 11_level_0  
     서스틴 베스트     Moody's    MSCI     S&P 지속가능경영 보고서공시여부 Unnamed: 11_level_1  
0   서스틴베스트 -   Moody's -  MSCI -  S&P 24            NaN     지속가능경영보고서 공시 여부  
1  서스틴베스트 BB   Moody's -  MSCI -  S&P 36            NaN     지속가능경영보고서 공시 여부  
2  서스틴베스트 BB   Moody's -  MSCI -  S&P 39            NaN     지속가능경영보고서 공시 여부  
3   서스틴베스트 -   Moody's -  MSCI -  S&P 29            NaN     지속가능경영보고서 공시 여부  
4   서스틴베스트 B   Moody's -  MSCI -   S&P 7            NaN     지속가능경영보고서 공시 여부  
5   서스틴베스트 A  Moody's 24  MSCI -  S&P 42            NaN     지속가능경영보고서 공시 여부  
6   서스틴베스트 -   Moody's -  MSCI -   S&P -            NaN     지속가능경영보고서 공시 여부  
7   서스틴베스트 -   Moody's -  MSCI -   S&P 6            NaN     지속가능경영보고서 공시 여부  
8   서스틴베스트 -   Moody's -  MSCI -   S&P -            NaN     지속가능경영보고서 공시 여부  
9   서스틴베스트 -   Moody's -  MSCI -   S&P -            NaN     지속가능경영보고서 공시 여부

In [7]:
# 리스트 안 데이터프레임들 통합후 필요 컬럼 추출
esg_raw_list = [table.droplevel(axis=1, level=0)[['회사명', '종합등급', '환경', '사회', '지배구조', '한국ESG 연구소', '서스틴 베스트', "Moody's",'MSCI','S&P']] 
                for table in esg_raw_data_list]
esg_raw_list[0]

,회사명,종합등급,환경,사회,지배구조,한국ESG 연구소,서스틴 베스트,Moody's,MSCI,S&P
0,AJ네트웍스 2020 N N,KCGS B,환경 D,사회 B,지배구조 B,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -
1,AK홀딩스 2020 N N,KCGS B+,환경 C,사회 B+,지배구조 B+,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -
2,BGF 2020 N N,KCGS B+,환경 B,사회 A,지배구조 B+,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -
3,BGF리테일 2020 N N,KCGS A,환경 B+,사회 B+,지배구조 A,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI BB,S&P -
4,BNK금융지주 2020 N N,KCGS A+,환경 B+,사회 A+,지배구조 A+,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -
5,BYC 2020 N N,KCGS B,환경 D,사회 B,지배구조 B+,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -
6,CJ 2020 Y Y,KCGS A,환경 B,사회 A,지배구조 A,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI BBB,S&P -
7,CJ CGV 2020 N N,KCGS A,환경 B,사회 A,지배구조 A,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -
8,CJ대한통운 2020 N N,KCGS A,환경 A,사회 B,지배구조 A,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI B,S&P -
9,CJ씨푸드 2020 N N,KCGS B,환경 C,사회 B+,지배구조 B+,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -


In [8]:
len(esg_raw_list)

314

In [11]:
# dataframe 통합
import pandas as pd
esg_raw_concat = pd.concat(esg_raw_list)
esg_raw_concat.tail()

,회사명,종합등급,환경,사회,지배구조,한국ESG 연구소,서스틴 베스트,Moody's,MSCI,S&P
5,휠라홀딩스 2023 N N,KCGS B+,환경 B,사회 A,지배구조 B+,한국ESG 연구소 A,서스틴베스트 A,Moody's 24,MSCI -,S&P 42
6,휴니드테크놀러지스 2023 N N,KCGS D,환경 D,사회 C,지배구조 C,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -
7,휴비스 2023 N N,KCGS B+,환경 B+,사회 A,지배구조 B+,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P 6
8,휴스틸 2023 N N,KCGS D,환경 D,사회 D,지배구조 C,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -
9,흥국화재 2023 N N,KCGS B+,환경 C,사회 B+,지배구조 B+,한국ESG 연구소 -,서스틴베스트 -,Moody's -,MSCI -,S&P -


In [31]:
# 데이터 추출
년도 = [each_name.split()[1:-2][-1]
        for each_name in esg_raw_concat['회사명']]
회사명 = [' '.join(each_name.split()[:-3]) for each_name in esg_raw_concat['회사명']]
종합등급 = [each_name.split()[-1] for each_name in esg_raw_concat['종합등급']]
환경 = [each_name.split()[-1] for each_name in esg_raw_concat['환경']]
사회 = [each_name.split()[-1] for each_name in esg_raw_concat['사회']]
지배구조 = [each_name.split()[-1] for each_name in esg_raw_concat['지배구조']]
한국ESG연구소 = [each_name.split()[-1] for each_name in esg_raw_concat['한국ESG 연구소']]
서스틴베스트 = [each_name.split()[-1] for each_name in esg_raw_concat['서스틴 베스트']]
Moody = [each_name.split()[-1] for each_name in esg_raw_concat["Moody's"]]
MSCI = [each_name.split()[-1] for each_name in esg_raw_concat['MSCI']]
SP = [each_name.split()[-1] for each_name in esg_raw_concat['S&P']]

In [32]:
# 데이터프레임 생성
esg_df = pd.DataFrame({
      '년도' : 년도 
    , '회사명' : 회사명
    , '종합등급' : 종합등급
    , '환경' : 환경
    , '사회' : 사회
    , '지배구조' : 지배구조
    , '한국ESG연구소' : 한국ESG연구소
    , '서스틴베스트' : 서스틴베스트
    , 'Moody' : Moody
    , 'MSCI' : MSCI
    , 'SP' : SP
})
esg_df.head()

,년도,회사명,종합등급,환경,사회,지배구조,한국ESG연구소,서스틴베스트,Moody,MSCI,SP
0,2020,AJ네트웍스,B,D,B,B,-,-,-,-,-
1,2020,AK홀딩스,B+,C,B+,B+,-,-,-,-,-
2,2020,BGF,B+,B,A,B+,-,-,-,-,-
3,2020,BGF리테일,A,B+,B+,A,-,-,-,BB,-
4,2020,BNK금융지주,A+,B+,A+,A+,-,-,-,-,-


In [33]:
# 데이터 확인 => OK
esg_df.tail(10)

,년도,회사명,종합등급,환경,사회,지배구조,한국ESG연구소,서스틴베스트,Moody,MSCI,SP
3130,2023,효성중공업,A,A,A+,B+,-,-,-,-,24
3131,2023,효성첨단소재,A,B+,A+,A,A,BB,-,-,36
3132,2023,효성티앤씨,A,A,A+,B+,A,BB,-,-,39
3133,2023,효성화학,A,A,A+,B+,-,-,-,-,29
3134,2023,후성,B,B,B+,B,B+,B,-,-,7
3135,2023,휠라홀딩스,B+,B,A,B+,A,A,24,-,42
3136,2023,휴니드테크놀러지스,D,D,C,C,-,-,-,-,-
3137,2023,휴비스,B+,B+,A,B+,-,-,-,-,6
3138,2023,휴스틸,D,D,D,C,-,-,-,-,-
3139,2023,흥국화재,B+,C,B+,B+,-,-,-,-,-


In [34]:
# 파일 저장
esg_df.to_csv('esg_grade.csv', index=False)